In [1]:
import os.path as path

from tqdm import tqdm
from joblib import delayed, Parallel, dump
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

from models.db_session import create_session, global_init
from models.news import News

from classes import Corpus, Document, Vectorizer

In [2]:
global_init(path.join('db', 'news.sqlite'))
session = create_session()

In [3]:
SEED = 0

In [4]:
corpus = Corpus()
print('Создаем корпус новостей...')
vectorizer = Vectorizer()
news = session.query(News).all()
data = zip(news, Parallel(n_jobs=-1)(
    delayed(vectorizer)(s.full_text)
    for s in tqdm(news)
))
for news, lemmas in data:
    corpus.add_document(Document(news=news, corpus=corpus, lemmas=lemmas))

Создаем корпус новостей...


100%|██████████| 1797/1797 [02:12<00:00, 13.58it/s]


In [5]:
len(corpus.lemmas)

17979

In [6]:
# Оставляем слова, которые встречаются не менее 5 раз

corpus.filter(min_freq=5)
len(corpus.lemmas)

4745

In [7]:
corpus.save(path.join('data', 'corpus.pkl'))

In [8]:
tf_idf = corpus.get_tf_idf(is_normalize=True)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf, corpus.categories_labels, test_size=0.15, random_state=SEED)

In [10]:
model = MultinomialNB(alpha=0.01)
model.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

In [11]:
predicted = model.predict(X_train)
print(metrics.classification_report(y_train, predicted, zero_division=np.NaN))

                   precision    recall  f1-score   support

   69-я параллель       1.00      1.00      1.00         3
      Бывший СССР       0.99      0.93      0.96       207
    Забота о себе       0.95      1.00      0.97        39
         Из жизни       1.00      1.00      1.00        51
   Интернет и СМИ       0.93      0.86      0.89        59
         Культура       0.92      1.00      0.96        66
              Мир       0.91      0.99      0.95       275
       Моя страна       1.00      1.00      1.00        15
  Наука и техника       1.00      0.97      0.98        67
      Путешествия       1.00      1.00      1.00        57
           Россия       0.98      0.93      0.95       262
Силовые структуры       1.00      1.00      1.00       124
            Спорт       1.00      0.98      0.99        62
   Среда обитания       1.00      1.00      1.00         5
         Ценности       1.00      1.00      1.00        53
        Экономика       0.98      0.98      0.98       

In [12]:
predicted = model.predict(X_test)
print(metrics.classification_report(y_test, predicted, zero_division=np.NaN))

                   precision    recall  f1-score   support

   69-я параллель        nan      0.00      0.00         1
      Бывший СССР       0.88      0.72      0.79        40
    Забота о себе       0.73      0.89      0.80         9
         Из жизни       0.30      0.60      0.40         5
   Интернет и СМИ       0.86      0.33      0.48        18
         Культура       0.83      0.71      0.77        14
              Мир       0.78      0.90      0.84        42
       Моя страна       1.00      0.67      0.80         3
  Наука и техника       0.56      0.71      0.62         7
      Путешествия       1.00      0.67      0.80         9
           Россия       0.74      0.89      0.81        35
Силовые структуры       0.83      0.89      0.86        27
            Спорт       1.00      1.00      1.00         5
   Среда обитания        nan      0.00      0.00         1
         Ценности       0.92      1.00      0.96        12
        Экономика       0.86      0.86      0.86       

In [13]:
dump(model, path.join('data', 'classifier.pkl'));